In [1]:
import matplotlib.pyplot as plt
import mplfinance
import numpy as np

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import datetime
import yfinance as yf
import time
import stockstats
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot, init_notebook_mode
from collections import OrderedDict
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
#grouping by frequency 
def FrameChanging(df, x):
    t = df.groupby(pd.Grouper(freq=x)).agg({"open": "first", 
                                             "close": "last", 
                                             "low": "min", 
                                             "high": "max",
                                             "volume": "sum"})
    return t

In [4]:
#adding column dateindex of data type datetime to set as index of data frame
def DatePre(df):
    df['DateIndex'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('DateIndex')
    df = df.drop(['timestamp'], axis=1)
    return df

In [5]:
#reading sample dataset for use case 
df = pd.read_csv("../Data/ADANIPOWER__EQ__NSE__NSE__MINUTE.csv")

In [6]:
df.head()

,timestamp,open,high,low,close,volume
0,2017-01-02 09:15:00+05:30,30.10,30.20,30.10,30.15,39761.0
1,2017-01-02 09:16:00+05:30,30.15,30.20,30.10,30.20,11081.0
2,2017-01-02 09:17:00+05:30,30.20,30.20,30.10,30.10,9706.0
3,2017-01-02 09:18:00+05:30,30.10,30.15,30.05,30.05,74384.0
4,2017-01-02 09:19:00+05:30,30.05,30.10,30.00,30.05,21735.0


In [7]:
#converting timestamp to datetime format 
df['DateIndex'] = pd.to_datetime(df['timestamp'])
df = df.set_index('DateIndex')
df = df.drop(['timestamp'], axis=1)

In [8]:
df.shape

(321361, 5)

In [9]:
# grouping the dataset for the interval of 30 min
df1 = FrameChanging(df, '30Min')

In [10]:
df1.shape

(61021, 5)

In [11]:
df1.head()

,open,close,low,high,volume
DateIndex,,,,,
2017-01-02 09:00:00+05:30,30.10,29.95,29.95,30.20,372228.0
2017-01-02 09:30:00+05:30,30.00,29.95,29.90,30.10,713049.0
2017-01-02 10:00:00+05:30,29.95,29.70,29.65,29.95,677971.0
2017-01-02 10:30:00+05:30,29.70,29.45,29.40,29.70,707812.0
2017-01-02 11:00:00+05:30,29.45,29.40,29.35,29.65,1473280.0


In [12]:
#moving average calculated over 100 200 and 50 days
df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']

In [13]:
df1.shape

(61021, 8)

In [14]:
df1.describe()

,open,close,low,high,volume,MA100,MA200,MA50
count,11140.000000,11140.000000,11140.000000,11140.000000,6.102100e+04,10285.000000,10285.000000,10285.000000
mean,40.310983,40.306113,40.068855,40.550058,2.241536e+05,40.301345,40.300124,40.301201
std,13.439303,13.436619,13.385378,13.498862,8.875962e+05,13.435373,13.431913,13.437123
min,15.250000,15.250000,15.150000,15.450000,0.000000e+00,15.369000,15.406500,15.349000
25%,30.300000,30.300000,30.137500,30.450000,0.000000e+00,30.269500,30.274250,30.276000
50%,36.350000,36.350000,36.100000,36.600000,0.000000e+00,36.354000,36.389750,36.366000
75%,50.512500,50.500000,50.300000,50.762500,0.000000e+00,50.513000,50.501250,50.514000
max,72.950000,72.950000,72.850000,73.800000,2.900877e+07,72.974000,72.654000,73.102000


In [15]:
df1 = df1.dropna()

In [16]:
C_DownTrend = True
C_UpTrend = True

In [17]:
#marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)

In [18]:
C_Len = 14 #ema depth for bodyAvg
C_ShadowPercent = 5.0 # size of shadows
C_ShadowEqualsPercent = 100.0
C_DojiBodyPercent = 5.0
C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body

In [19]:
#finding body high and low and body length from closing and opening price of the day
df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']

In [22]:
df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()

In [24]:
#setting C_SmallBody as 1 if body average is > the body length and 0 if body average is <= to body length
df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = '1'
df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = '0'

#setting C_LongBody as 1 if body average is < the body length and 0 if body average is >= to body length
df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = '1'
df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = '0'

In [25]:
#calculating upshadow as high - body high and down shadow as body low - low
df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']

In [27]:
#setting C_HasUpShadow as 0 or 1 indicating presence of up shadow
df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     "1", "0")

In [28]:
#setting C_HasDnShadow as 0 or 1 indicating presence of down shadow
df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     "1", "0")

In [29]:
#white body reprecents traditional green candlestick and black represents red
df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)

In [30]:
df1['C_Range'] = df1['high']-df1['low']

In [31]:
df1['C_IsInsideBar'] =  np.where( ((df1.C_BodyHi.shift(1)  < df1.C_BodyHi) & ( df1.C_BodyLo.shift(1) < df1.C_BodyLo ) ),"1", "0")

In [32]:
df1['C_IsInsideBar'].value_counts()

0    8146
1    2137
Name: C_IsInsideBar, dtype: int64

In [33]:
df1['C_BodyMiddle'] = df1['C_Body'] / (2 + df1['C_BodyLo'])

In [34]:
#findng if upshadow is equal to dowm shadow
df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)

In [35]:
df1['C_ShadowEquals'].value_counts()

True     8536
False    1747
Name: C_ShadowEquals, dtype: int64

In [36]:
#checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)

In [37]:
df1['C_IsDojiBody'].value_counts()

False    9017
True     1266
Name: C_IsDojiBody, dtype: int64

In [38]:
df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']

In [39]:
#checking if it is an abandoned baby bearish pattern and marking as 1 if yes and 0 if no
df1['C_AbandonedBabyBearish'] = np.where( ( ( df1['C_UpTrend'].shift(2)  & df1['C_WhiteBody'].shift(2) & 
                                             df1['C_IsDojiBody'].shift(1) &  (df1['high'].shift(2) <df1['low'].shift(1)) &
                                             df1['C_BlackBody'] & (df1['low'].shift(1) > df1['high'] )   ) )    , "1", "0" )

In [40]:
#counting the number of abandoned baby bearish pattern
df1['C_AbandonedBabyBearish'].value_counts()

0    10283
Name: C_AbandonedBabyBearish, dtype: int64